# CNN on MNIST (Simple)

In [3]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

#### Helper functions

In [21]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

def conv_layer(input, shape):
    W = weight_variable(shape)
    b = bias_variable([shape[3]])
    return tf.nn.relu(conv2d(input, W) + b)

def full_layer(input, size):
    in_size = int(input.get_shape()[1])
    W = weight_variable([in_size, size])
    b = bias_variable([size])
    return tf.matmul(input, W) + b

#### Input data

In [8]:
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


#### Model

In [22]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(x, [-1, 28, 28, 1])

conv_1 = conv_layer(x_image, shape=[5, 5, 1, 32])
max_pool_1 = max_pool_2x2(conv_1)

conv_2 = conv_layer(max_pool_1, [5, 5, 32, 64])
max_pool_2 = max_pool_2x2(conv_2)

conv_2_flat = tf.reshape(max_pool_2, [-1, 7*7*64])
full_1 = tf.nn.relu(full_layer(conv_2_flat, 1024))

keep_prob = tf.placeholder(tf.float32)
full_1_drop = tf.nn.dropout(full_1, keep_prob=keep_prob)

y_conv = full_layer(full_1_drop, 10)

In [24]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

optimizer = tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [35]:
NUM_STEPS = 5000
MINI_BATCH_SIZE = 50
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for i in range(NUM_STEPS):
        batch_x, batch_y = mnist.train.next_batch(MINI_BATCH_SIZE)
        
        if i%100 == 0:
            train_accuracy = sess.run(accuracy, feed_dict={
                x: batch_x, y_: batch_y, keep_prob: 1.0
            })
            print "step: {} , training_accuracy: {}".format(i, train_accuracy)
        
        sess.run(optimizer, feed_dict={
                x: batch_x, y_: batch_y, keep_prob: 0.5
            })
    
    X_test = mnist.test.images.reshape(10, 1000, 784)
    y_test = mnist.test.labels.reshape(10, 1000, 10)
    test_accuracy = np.mean([sess.run(accuracy,
                                     feed_dict= {x: X_test[i], y_: y_test[i], keep_prob: 1.0})
                            for i in range(10)])
    print "test accuracy: {}".format(train_accuracy)

step: 0 , training_accuracy: 0.0799999982119
step: 100 , training_accuracy: 0.839999973774
step: 200 , training_accuracy: 0.899999976158
step: 300 , training_accuracy: 0.939999997616
step: 400 , training_accuracy: 0.980000019073
step: 500 , training_accuracy: 0.899999976158
step: 600 , training_accuracy: 0.959999978542
step: 700 , training_accuracy: 0.939999997616
step: 800 , training_accuracy: 0.879999995232
step: 900 , training_accuracy: 0.980000019073
step: 1000 , training_accuracy: 0.959999978542
step: 1100 , training_accuracy: 1.0
step: 1200 , training_accuracy: 0.959999978542
step: 1300 , training_accuracy: 0.939999997616
step: 1400 , training_accuracy: 0.939999997616
step: 1500 , training_accuracy: 0.980000019073
step: 1600 , training_accuracy: 1.0
step: 1700 , training_accuracy: 0.980000019073
step: 1800 , training_accuracy: 1.0
step: 1900 , training_accuracy: 1.0
step: 2000 , training_accuracy: 1.0
step: 2100 , training_accuracy: 0.980000019073
step: 2200 , training_accuracy: 